# Self-supervised Learning
The objective of this lab project is to go further in the understanding of Self-Supervised Learning (SSL). By the end of the notebook, you will
- Train models using different pretext tasks: colorizing, inpainting, masking reconstruction.
- Fine-tune the models with the downstream task of interst.
- Compare the performance of the different backbones obtained from the different downstream tasks.

## Library imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

import zipfile
import shutil
import os


## Pretext and Downstream Tasks

We will train three different models using three different pretext tasks. All three models will be trained on the SVHN dataset. The three models are the following:
- A Colorization Neural Network
- An Inpainting Neural Network
- A Masked Autoencoder

We will build a common architecture so that all three models have as similar architectures as possible. The common architecture will consist of an encoder and a decoder. Once the models pre-trained on their respective pretext tasks, we will use the pre-trained encoders to evaluate the learnt representations on two new downstream task: image classification on MNIST and on SVHN. To that end, we will perform a linear evaluation protocol, by freezing the weights of the pre-trained encoders, and training a linear classifier on the learnt representations. 

# Hyperparameters

In [2]:
BATCH_SIZE = 128
RESIZE = 64
LATENT_DIM = 64

## Data Preparation

Download the datasets

In [7]:
# Crée un chemin relatif basé sur le dossier actuel
dataset_path = os.path.join(os.getcwd(), '../data/')

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path, exist_ok=True)  # Crée le dossier et ses parents si nécessaire

    !git clone https://github.com/sourisimos/DATA_P3HDDL_MVTEC.git ../data/mvtec #Cloning mvtec dataset 
    !git clone https://github.com/sourisimos/DATA_P3HDDL_EW.git ../data/ew #Cloning ew datasets

    print(f"Dossiers téléchargés dans le dossier '{dataset_path}'.")


Clonage dans '../data/mvtec'...
remote: Enumerating objects: 1287, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1287 (delta 2), reused 0 (delta 0), pack-reused 1275 (from 1)
Réception d'objets: 100% (1287/1287), 60.20 Mio | 9.03 Mio/s, fait.
Résolution des deltas: 100% (2/2), fait.
Clonage dans '../data/ew'...
remote: Enumerating objects: 857, done.
remote: Counting objects: 100% (857/857), done.
remote: Compressing objects: 100% (855/855), done.
remote: Total 857 (delta 3), reused 842 (delta 0), pack-reused 0 (from 0)
Réception d'objets: 100% (857/857), 15.80 Mio | 7.69 Mio/s, fait.
Résolution des deltas: 100% (3/3), fait.
Dossiers téléchargés dans le dossier '/home/hp/Scolarite/INSA/5A/HDDL/Projets_HDDL/P3_SSL_anomaly/code/../data/'.


## Engine wiring dataset

In [77]:
# Define the transformations to apply to EW

transform_ew_val = transforms.Compose([
    transforms.Resize((RESIZE, RESIZE)),
    transforms.ToTensor()

])

transform_ew_train = transforms.Compose([
    transforms.Resize((RESIZE,RESIZE)),
    transforms.RandomRotation(degrees=30),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()
])


# Load EW dataset for train test and creating val

# Training
ew_train_dataset = datasets.ImageFolder(root='../data/engine_wiring/train', transform=transform_ew_train)
ew_train_loader = DataLoader(ew_train_dataset, batch_size=BATCH_SIZE, shuffle=True )

# Test
ew_test_dataset = datasets.ImageFolder(root='../data/engine_wiring/test_merge', transform=transform_ew_val) 
ew_test_loader = DataLoader(ew_test_dataset, batch_size=BATCH_SIZE, shuffle=True)


# Device configuration

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## MVtec datasets

In [45]:
# Define the transformations to apply to the images
transform_mvtec = transforms.Compose([
    transforms.Resize((128, 128)),  
    transforms.ToTensor(),  
])

# Function to load the dataset mvtec 
def load_mvt_dataset(root='../data/mvtec_anomaly_detection', batch_size=64, restricted = True):
    # List of categories you want to load
    categories = ['bottle', 'cable','capsule', 'carpet', 'grid','hazelnut', 'leather', 'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper']  # Add other categories as needed
    # Restricted are just the essential one (for the academ project )
    if restricted: 
        categories = ['bottle','capsule', 'hazelnut', 'toothbrush']
        
    # Create dictionaries to hold loaders for training and test sets
    train_loaders = {}
    test_loaders = {}
    
    for category in categories:

        train_dataset = datasets.ImageFolder(root=f'{root}/{category}/train', transform=transform_mvtec)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        train_loaders[category] = train_loader
        
        test_dataset = datasets.ImageFolder(root=f'{root}/{category}/test', transform=transform_mvtec)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        test_loaders[category] = test_loader
        
    return train_loaders, test_loaders

train_loaders, test_loaders = load_mvt_dataset()

# Accessing a specific category's data loader
bottle_train_loader = train_loaders['bottle']
bottle_test_loader = test_loaders['bottle']

## Shared architecture
We will create a shared architecture with the following layers:
- The encoder (in sequential order):
    - A convolution with 64 filters, kernel size 4, stride 2, padding 1
    - A ReLU activation
    - A convolution with 128 filters, kernel size 4, stride2, padding 1
    - A ReLU activation
    - A convolution with `latent_dim`, kernel size 4, stride 2, padding 1
    - A ReLU activation
The encoder should take the number of channels of the input `in_channels` and the hidden dimension `latent_dim` as arguments.
- The decoder (in secuential ) order:
    - A Transpose convolution with 128 filters, kernel size 4, padding 1
    - A ReLU activation
    - A convolution with 16428 filters, kernel size 4, stride2, padding 1
    - A ReLU activation
    - A convolution with `out_channels` filters, kernel size 4, stride 2, padding 1
    - A ReLU activation
The encoder should take the number of channels of the output `out_channels` and the hidden dimension `latent_dim` as arguments.

In [78]:
class Encoder(nn.Module):
    def __init__(self, in_channels=3, latent_dim=32):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1),  # 32x32 -> 16x16
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # 16x16 -> 8x8
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # 8x8 -> 4x4
            nn.ReLU(),
            nn.Conv2d(256, latent_dim, kernel_size=4, stride=2, padding=1),  # 4x4 -> 2x2
            nn.ReLU(),
        )

    def forward(self, x):
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim=32, out_channels=3):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 256, kernel_size=4, stride=2, padding=1),  # 2x2 -> 4x4
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 4x4 -> 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 8x8 -> 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(64, out_channels, kernel_size=4, stride=2, padding=1),  # 16x16 -> 32x32
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.decoder(x)

In [79]:
def train_ssl_model(model, 
                    train_loader, 
                    val_loader, 
                    criterion,
                    optimizer,
                    device=device,
                    epochs=5):

    for epoch in range(epochs):
        model.to(device)
        model.train()
        total_train_loss = 0
        for images, _ in train_loader:
            images = images.to(device)
            optimizer.zero_grad()
            output, _ = model(images)
            loss = criterion(output, images)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for images, _ in val_loader:
                images = images.to(device)
                output, _ = model(images)
                val_loss = criterion(output, images)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")
    

    return model.encoder

# Colorization model 

a de grande chance d'être nul dans le cadre de nos données

In [80]:
class ColorizationModel(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super(ColorizationModel, self).__init__()
        self.encoder = Encoder(latent_dim=latent_dim, in_channels=1)  # Input grayscale
        self.decoder = Decoder(latent_dim=latent_dim, out_channels=3)  # Predict RGB

    def forward(self, x):
        grayscale_x = transforms.Grayscale()(x)  # Convert RGB to Grayscale
        z = self.encoder(grayscale_x)
        return self.decoder(z), grayscale_x

In [81]:
colorization_model = ColorizationModel(latent_dim=LATENT_DIM)
colorization_encoder = train_ssl_model(colorization_model, 
                                       ew_train_loader, 
                                       ew_test_loader, 
                                       criterion=nn.MSELoss(), 
                                       optimizer=optim.Adam(colorization_model.parameters(), lr=0.001)
                                       )

Epoch 1/5, Train Loss: 0.1077, Avg Val Loss: 0.0969
Epoch 2/5, Train Loss: 0.1045, Avg Val Loss: 0.0964
Epoch 3/5, Train Loss: 0.1033, Avg Val Loss: 0.0808
Epoch 4/5, Train Loss: 0.0835, Avg Val Loss: 0.0680
Epoch 5/5, Train Loss: 0.0663, Avg Val Loss: 0.0548


# Inpainting model 

In [82]:
class InpaintingModel(nn.Module):
    def __init__(self, latent_dim=128, mask_size=8):
        super(InpaintingModel, self).__init__()
        self.encoder = Encoder(latent_dim=latent_dim)
        self.decoder = Decoder(latent_dim=latent_dim)
        self.mask_size = mask_size

    def forward(self, x):
        x_masked = self.apply_mask(x)
        z = self.encoder(x_masked)
        return self.decoder(z), x_masked
    
    def apply_mask(self, x):
        masked_x = x.clone()

        for i in range(masked_x.size(0)):
            ul_x = np.random.randint(0, x.size(2) - self.mask_size + 1)
            ul_y = np.random.randint(0, x.size(3) - self.mask_size + 1)
            masked_x[i, :, ul_x:ul_x+self.mask_size, ul_y:ul_y+self.mask_size] = 0

        return masked_x

In [83]:
inpainting_model = InpaintingModel(latent_dim=LATENT_DIM, mask_size=8)
inpainting_encoder = train_ssl_model(inpainting_model,
                                     ew_train_loader, 
                                     ew_test_loader, 
                                     criterion=nn.MSELoss(), 
                                     optimizer=optim.Adam(inpainting_model.parameters(), lr=0.001)
                                     )

Epoch 1/5, Train Loss: 0.1058, Avg Val Loss: 0.0977
Epoch 2/5, Train Loss: 0.1055, Avg Val Loss: 0.0902
Epoch 3/5, Train Loss: 0.0914, Avg Val Loss: 0.0632
Epoch 4/5, Train Loss: 0.0896, Avg Val Loss: 0.0764
Epoch 5/5, Train Loss: 0.0792, Avg Val Loss: 0.0642


# Masked AE Model

In [84]:
class MaskedAutoencoderModel(nn.Module):
    def __init__(self, latent_dim=32, mask_ratio=1/16):
        super(MaskedAutoencoderModel, self).__init__()
        self.encoder = Encoder(latent_dim=latent_dim)
        self.decoder = Decoder(latent_dim=latent_dim)
        self.mask_ratio = mask_ratio

    def forward(self, x):
        x_masked = self.apply_mask(x)
        z = self.encoder(x_masked)
        return self.decoder(z), x_masked
    
    def apply_mask(self, x):
        x_masked = x.clone()
        mask = torch.rand_like(x[:, 0, :, :]) < self.mask_ratio
        mask = mask.unsqueeze(1).repeat(1, x.size(1), 1, 1)
        x_masked[mask] = 0
        return x_masked

In [85]:
mae_model = MaskedAutoencoderModel(latent_dim=LATENT_DIM, mask_ratio=1/16)
mae_encoder = train_ssl_model(mae_model,
                              ew_train_loader, 
                              ew_train_loader, 
                              criterion=nn.MSELoss(), 
                              optimizer=optim.Adam(mae_model.parameters(), lr=0.001)
                              )

Epoch 1/5, Train Loss: 0.1073, Avg Val Loss: 0.1054
Epoch 2/5, Train Loss: 0.1008, Avg Val Loss: 0.0960
Epoch 3/5, Train Loss: 0.0888, Avg Val Loss: 0.0772
Epoch 4/5, Train Loss: 0.0750, Avg Val Loss: 0.0640
Epoch 5/5, Train Loss: 0.0635, Avg Val Loss: 0.0632


# Contrastive Model

In [107]:
# Define the Contrastive Dataset
class ContrastiveDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Get original image
        img, _ = self.dataset[idx]
        # Create positive sample through augmentation
        return img.float(), img.float()


# Define the Projection Head
class ProjectionHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ProjectionHead, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Define the Contrastive Loss
class ContrastiveLoss(nn.Module):
    def __init__(self, temperature=0.1):
        super(ContrastiveLoss, self).__init__()
        self.temperature = temperature

    def forward(self, z1, z2):
        z1 = nn.functional.normalize(z1, dim=-1)
        z2 = nn.functional.normalize(z2, dim=-1)
        similarity = torch.mm(z1, z2.T) / self.temperature
        labels = torch.arange(z1.size(0)).to(z1.device)
        loss = nn.functional.cross_entropy(similarity, labels)
        return loss

In [110]:
# Training loop for contrastive learning
def train_contrastive_model(encoder, projection_head, 
                    criterion,
                    optimizer, train_loader, val_loader, epochs=5):

    for epoch in range(epochs):
        encoder.train()
        projection_head.train()
        total_loss = 0
        
        for img1, img2 in train_loader:
            img1, img2 = img1.to(device), img2.to(device)

            optimizer.zero_grad()
            z1 = encoder(img1)
            z1 = projection_head(z1.view(z1.size(0), -1))  # Aplatir pour la tête de projection
            z2 = encoder(img2)
            z2 = projection_head(z2.view(z2.size(0), -1))

            loss = criterion(z1, z2)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        encoder.eval()
        total_val_loss = 0
        with torch.no_grad():
            for images, _ in val_loader:
                images = images.to(device)
                output, _ = encoder(images)
                val_loss = criterion(output, images)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")
        

In [111]:
# Train the contrastive model

encoder = Encoder(in_channels=3, latent_dim=LATENT_DIM).to(device)
projection_head = ProjectionHead(input_dim=LATENT_DIM * 4 * 4, output_dim=64).to(device)
contrastive_loss = ContrastiveLoss().to(device)
optimizer = optim.Adam(list(encoder.parameters()) + list(projection_head.parameters()), lr=1e-3)

train_contrastive_model(encoder, projection_head, contrastive_loss, optimizer ,ew_train_loader, ew_test_loader, epochs=5)


ValueError: too many values to unpack (expected 2)

# Anomaly detection 

In [86]:
class Anomaly_detector(nn.Module):
    def __init__(self, encoder, num_classes=2):
        super(Anomaly_detector, self).__init__()
        self.encoder = encoder
        self.fc = nn.Linear(4*4*LATENT_DIM, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        z = z.view(z.size(0), -1)
        return self.fc(z)

In [94]:
# Fine-tuning loop for classification
def fine_tune_ew(encoder, train_loader, test_loader, epochs=5, encoder_in_channels=3):
    
    model = Anomaly_detector(encoder).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-5)
    criterion = nn.CrossEntropyLoss()
    # Freeze the encoder's weights
    for param in model.encoder.parameters():
        param.requires_grad = False
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for images, labels in train_loader:
            if encoder_in_channels == 1:
                images = transforms.Grayscale()(images)  # Convert RGB to Grayscale
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            output = model(images)
            # print('output:', output)
            # print(np.shape(output))

            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            # Compute accuracy
            _, predicted = torch.max(output, 1)
            # print("\n predicted", predicted, '\n')
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}, Accuracy: {100 * correct / total:.2f}%")
    
    
    # Evaluate on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            if encoder_in_channels == 1:
                images = transforms.Grayscale()(images)  # Convert RGB to Grayscale
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            # print(output)
            # print(np.shape(output))

            _, predicted = torch.max(output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            print(predicted)
    print(f'Test Accuracy: {100 * correct / total:.2f}%')

# Résults

In [95]:
fine_tune_ew(colorization_encoder, ew_train_loader, ew_test_loader, encoder_in_channels=1)

Epoch 1/5, Loss: 0.18128189941247305, Accuracy: 100.00%
Epoch 2/5, Loss: 0.16985451181729636, Accuracy: 100.00%
Epoch 3/5, Loss: 0.16324039796988168, Accuracy: 100.00%
Epoch 4/5, Loss: 0.14742258687814078, Accuracy: 100.00%
Epoch 5/5, Loss: 0.13639810432990393, Accuracy: 100.00%
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [96]:
fine_tune_ew(inpainting_encoder, ew_train_loader, ew_test_loader, encoder_in_channels=3)

Epoch 1/5, Loss: 0.5533777872721354, Accuracy: 62.11%
Epoch 2/5, Loss: 0.4865899582703908, Accuracy: 61.40%
Epoch 3/5, Loss: 0.48490514357884723, Accuracy: 63.16%
Epoch 4/5, Loss: 0.5186337828636169, Accuracy: 69.47%
Epoch 5/5, Loss: 0.44235386451085407, Accuracy: 77.19%
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [97]:
fine_tune_ew(mae_encoder, ew_train_loader, ew_test_loader, encoder_in_channels=3)

Epoch 1/5, Loss: 0.9703331391016642, Accuracy: 36.14%
Epoch 2/5, Loss: 0.9385581016540527, Accuracy: 37.89%
Epoch 3/5, Loss: 0.9038851857185364, Accuracy: 38.95%
Epoch 4/5, Loss: 0.9110100467999777, Accuracy: 40.70%
Epoch 5/5, Loss: 0.8556139866511027, Accuracy: 40.00%
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0